In [1]:
import requests
import utils
import json
import random
import os
import pandas as pd

from tqdm.notebook import tqdm
from itertools import chain

# 1) a new zero shot classifier is created with a custom label

In [2]:
category = 'dark'
r = requests.post(f"http://127.0.0.1:8000/api/zero-shot/?category={category}")
print(r.url)
r.status_code

http://127.0.0.1:8000/api/zero-shot/?category=dark


200

# 2) a for loop iterates over a file with paragraphs and sends each paragraph to the classifier
# 3) the decision and confidence of the classifier is stored together with the paragraph in objects.

In [3]:
# inter loop for booksummaries demo-dataset: explotes 'Plot' column of the df and then classify each paraf depending on thresh

training_text = utils.get_training_text()

responses_infer = []
for ix, sample_text in tqdm(enumerate(training_text)):
    payload = {
      "single_inference": "true",
      "description": "zero-shot using BART-MNLI",
      "paragraph_size": 40,
      "threshold": 60,
      "text": sample_text
    }
    print(f"classifying --> {payload['text'][0:50]+' ...'}")
    r = requests.post("http://127.0.0.1:8000/api/zero-shot/1", data = json.dumps(payload))
    print(r.url)
    print(f"{ix+1}: status_code : {r.status_code}\n")
    responses_infer.append(r.json())

lenght before trimmering shorties: 9293
lenght after trimmering shorties: 9278
training_text lenght: 186


classifying -->  The seemingly immortal God Emperor Leto II has ru ...
http://127.0.0.1:8000/api/zero-shot/1
1: status_code : 200

classifying --> His continued evolution has slowly transformed him ...
http://127.0.0.1:8000/api/zero-shot/1
2: status_code : 200

classifying --> It is later revealed that his brain has gradually  ...
http://127.0.0.1:8000/api/zero-shot/1
3: status_code : 200

classifying --> During his long reign, Leto has enforced a state o ...
http://127.0.0.1:8000/api/zero-shot/1
4: status_code : 200

classifying --> The Bene Gesserit and Spacing Guild have endured,  ...
http://127.0.0.1:8000/api/zero-shot/1
5: status_code : 200

classifying --> Letos enforced peace brings stagnation to the gala ...
http://127.0.0.1:8000/api/zero-shot/1
6: status_code : 200

classifying --> Few people realize the burden that he carries as h ...
http://127.0.0.1:8000/api/zero-shot/1
7: status_code : 200

classifying --> As his father before him, Leto is utterly incapabl ...
http://127.0

http://127.0.0.1:8000/api/zero-shot/1
63: status_code : 200

classifying --> But Rhett stops her and pins her shoulders to the  ...
http://127.0.0.1:8000/api/zero-shot/1
64: status_code : 200

classifying --> Rhett then takes Scarlett in his arms and carries  ...
http://127.0.0.1:8000/api/zero-shot/1
65: status_code : 200

classifying --> She learns she is pregnant with her fourth child.  ...
http://127.0.0.1:8000/api/zero-shot/1
66: status_code : 200

classifying --> Scarlett tells him she is pale because she is preg ...
http://127.0.0.1:8000/api/zero-shot/1
67: status_code : 200

classifying --> At that comment, Scarlett lunges at Rhett, but he  ...
http://127.0.0.1:8000/api/zero-shot/1
68: status_code : 200

classifying --> Sobbing and drunk, Rhett buries his head in Melani ...
http://127.0.0.1:8000/api/zero-shot/1
69: status_code : 200

classifying --> When she returns a healthy woman to Atlanta, she s ...
http://127.0.0.1:8000/api/zero-shot/1
70: status_code : 200

classifying -->

http://127.0.0.1:8000/api/zero-shot/1
125: status_code : 200

classifying --> To a clean-shaven man who had an official manner,  ...
http://127.0.0.1:8000/api/zero-shot/1
126: status_code : 200

classifying --> To a journalist he gave a Report for publication,  ...
http://127.0.0.1:8000/api/zero-shot/1
127: status_code : 200

classifying --> The girl came forward, dressed in black, and met M ...
http://127.0.0.1:8000/api/zero-shot/1
128: status_code : 200

classifying --> Marlow envisioned them together: I saw her and him ...
http://127.0.0.1:8000/api/zero-shot/1
129: status_code : 200

classifying --> Being very uncomfortable Marlow tells her that the ...
http://127.0.0.1:8000/api/zero-shot/1
130: status_code : 200

classifying --> I knew it—I was sure!... She knew. She was sure. I ...
http://127.0.0.1:8000/api/zero-shot/1
131: status_code : 200

classifying --> It would have been too dark—too dark altogether... ...
http://127.0.0.1:8000/api/zero-shot/1
132: status_code : 200

classif

# 4) after the loop, random samples of the stored objects are printed

In [4]:
# For demo this is just printing the data['output'], used to compare to after fine-tuning results
# but I should cal a get request here

infer_output = []
for r_json in responses_infer:
    data = r_json['data']
    print(data, "\n")
    infer_output.append(data['output'])

{'classifier': '<transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x00000156B2059E88>', 'category': 'dark', 'output': '0.2905037999153137', 'message': 'Sucess! data stored to id_feed: 1~. Use this for adding training data.'} 

{'classifier': '<transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x00000156B2059E88>', 'category': 'dark', 'output': '0.6369053721427917', 'message': 'Sucess! data stored to id_feed: 1~. Use this for adding training data.'} 

{'classifier': '<transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x00000156B2059E88>', 'category': 'dark', 'output': '0.4551851451396942', 'message': 'Sucess! data stored to id_feed: 1~. Use this for adding training data.'} 

{'classifier': '<transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x00000156B2059E88>', 'category': 'dark', 'output': '0.671314537525177', 'message': 'Sucess! 

# 5) training data consisting of (paragraph, label) pairs are given to the classifier.

In [5]:
rounded_training_pairs = utils.generate_labeled_data()

lenght before trimmering shorties: 9293
lenght after trimmering shorties: 9278
training_text lenght: 186
classifying -->  The seemingly immortal God Emperor Leto II has ru ...
http://127.0.0.1:8000/api/zero-shot/1
1: status_code : 200

classifying --> His continued evolution has slowly transformed him ...
http://127.0.0.1:8000/api/zero-shot/1
2: status_code : 200

classifying --> It is later revealed that his brain has gradually  ...
http://127.0.0.1:8000/api/zero-shot/1
3: status_code : 200

classifying --> During his long reign, Leto has enforced a state o ...
http://127.0.0.1:8000/api/zero-shot/1
4: status_code : 200

classifying --> The Bene Gesserit and Spacing Guild have endured,  ...
http://127.0.0.1:8000/api/zero-shot/1
5: status_code : 200

classifying --> Letos enforced peace brings stagnation to the gala ...
http://127.0.0.1:8000/api/zero-shot/1
6: status_code : 200

classifying --> Few people realize the burden that he carries as h ...
http://127.0.0.1:8000/api/zero-shot/1


http://127.0.0.1:8000/api/zero-shot/1
62: status_code : 200

classifying --> Rhett, more drunk than Scarlett has ever seen him, ...
http://127.0.0.1:8000/api/zero-shot/1
63: status_code : 200

classifying --> But Rhett stops her and pins her shoulders to the  ...
http://127.0.0.1:8000/api/zero-shot/1
64: status_code : 200

classifying --> Rhett then takes Scarlett in his arms and carries  ...
http://127.0.0.1:8000/api/zero-shot/1
65: status_code : 200

classifying --> She learns she is pregnant with her fourth child.  ...
http://127.0.0.1:8000/api/zero-shot/1
66: status_code : 200

classifying --> Scarlett tells him she is pale because she is preg ...
http://127.0.0.1:8000/api/zero-shot/1
67: status_code : 200

classifying --> At that comment, Scarlett lunges at Rhett, but he  ...
http://127.0.0.1:8000/api/zero-shot/1
68: status_code : 200

classifying --> Sobbing and drunk, Rhett buries his head in Melani ...
http://127.0.0.1:8000/api/zero-shot/1
69: status_code : 200

classifying -->

http://127.0.0.1:8000/api/zero-shot/1
124: status_code : 200

classifying --> Better his cry—much better. Upon Marlows return to ...
http://127.0.0.1:8000/api/zero-shot/1
125: status_code : 200

classifying --> To a clean-shaven man who had an official manner,  ...
http://127.0.0.1:8000/api/zero-shot/1
126: status_code : 200

classifying --> To a journalist he gave a Report for publication,  ...
http://127.0.0.1:8000/api/zero-shot/1
127: status_code : 200

classifying --> The girl came forward, dressed in black, and met M ...
http://127.0.0.1:8000/api/zero-shot/1
128: status_code : 200

classifying --> Marlow envisioned them together: I saw her and him ...
http://127.0.0.1:8000/api/zero-shot/1
129: status_code : 200

classifying --> Being very uncomfortable Marlow tells her that the ...
http://127.0.0.1:8000/api/zero-shot/1
130: status_code : 200

classifying --> I knew it—I was sure!... She knew. She was sure. I ...
http://127.0.0.1:8000/api/zero-shot/1
131: status_code : 200

classif

http://127.0.0.1:8000/api/zero-shot/1
186: status_code : 200



In [6]:
# assign the training data to a classifier

rounded_training_pairs = [list(pair) for pair in rounded_training_pairs]

payload = {'pairs': rounded_training_pairs}

json.dumps(payload)

r = requests.put("http://127.0.0.1:8000/api/zero-shot/feedback/1", data = json.dumps(payload)) # make post for this

print(r.url)
r.status_code

http://127.0.0.1:8000/api/zero-shot/feedback/1


200

# 6) a retrain call is made

In [7]:
classifier_id = 1
r = requests.post(f"http://127.0.0.1:8000/api/zero-shot/retrain/{classifier_id}")
print(r.url)
print(r.status_code)
r.json()

http://127.0.0.1:8000/api/zero-shot/retrain/1
200


{'data': None,
 'message': 'Fine-tuned Model apended to the database under the id: 2. Try the new resource here --> http://127.0.0.1:8000/api/zero-shot/2'}

# 7) steps 2+3 again

In [8]:
# inter loop for booksummaries demo-dataset: explotes 'Plot' column of the df and then classify each paraf depending on thresh

payload = {
  "single_inference": "false",
  "description": "zero-shot using BART-MNLI",
  "paragraph_size": 40,
  "threshold": 65,
  "text": ""
}

r = requests.post("http://127.0.0.1:8000/api/zero-shot/2", data = json.dumps(payload))
print(r.url)
print(f"status_code : {r.status_code}")

r_json = r.json()
data = r_json['data']
output_2 = data['output']

http://127.0.0.1:8000/api/zero-shot/2
status_code : 200


# 8) compare the new classification scores and print the ones which changed the most

In [9]:
_, df_compare = utils.compare(infer_output, output_2, rounded_training_pairs)
df_compare

,text,train_label,change_abs,change
156,"At first, Huck is conflicted over whether to t...",1.0,0.350143,-35.014%
137,Bene Gesserit Imprinter Lucilla is also sent b...,1.0,0.262321,-26.232%
106,Mr. Kurtz couldnt be mad. If I had heard him t...,1.0,0.251295,-25.129%
86,All trying to better position themselves in a ...,1.0,0.248307,-24.831%
180,Jim is being held at the plantation of Silas a...,1.0,0.236326,-23.633%
...,...,...,...,...
69,"When she returns a healthy woman to Atlanta, s...",0.0,0.002843,-0.284%
108,The pilgrims carried Kurtz to the steamer and ...,0.0,0.002842,-0.284%
131,It would have been too dark—too dark altogethe...,1.0,0.002763,-0.276%
74,Mr. Butler tumbles to the ground then scramble...,1.0,0.002449,-0.245%
